In [1]:
import numpy as np
import json


In [25]:
import json
input_file = "nq-dev-kilt.jsonl"
output_file = "gold-nq-dev-kilt.json"

# print('total num documents = 5903530')
num_par_docs = 0
gold_data_list = []
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for l_id, line in enumerate(infile):
        if (l_id%1000==0):
            print(l_id)
        data = json.loads(line)
        new_data = {}
        new_data['id'] = data['id']
        # print(data)
        new_data['output'] = {'answer_set' : set(), \
                              'wiki_id_set' : set(), \
                                'wiki_par_id_set': set()}
        
        for p in data['output']:
            ans = p.get('answer', None)
            if (ans):
                new_data['output']['answer_set'].add(ans)
            prov = p.get('provenance', None)
            if prov:
                prov = prov[0]
                wiki_id = prov.get('wikipedia_id', None) 
                start_par_id = prov.get('start_paragraph_id', None) 
                end_par_id = prov.get('end_paragraph_id', None) 
                if(wiki_id):
                    new_data['output']['wiki_id_set'].add(wiki_id)
                if (start_par_id and end_par_id):
                    for p_id in range(start_par_id, end_par_id+1):
                        new_data['output']['wiki_par_id_set'].add(wiki_id + '_' + str(p_id))
        for k in new_data['output']:
            new_data['output'][k] = list(new_data['output'][k])
        # outfile.write(json.dumps(new_data) + "\n")
        gold_data_list.append(new_data)
        # break
print('done')
# print('num_par_docs = ', num_par_docs)

0
1000
2000
done


In [26]:
with open(output_file, 'w') as file:
    # Use json.dump to write the list of dictionaries to the file
    json.dump(gold_data_list, file, indent=4)